In [5]:
import asyncio
import json

from grpclib.client import Channel
from cortexadmin_pb import CortexAdminStub, SeriesRequest, QueryRangeRequest, QueryRequest, MatcherRequest
from betterproto.lib.google.protobuf import Empty


async def main():
  channel = Channel(host="localhost", port=11090) # url of opni-internal. can port-forward to localhost:11090
  service = CortexAdminStub(channel)
  # response = await service.get_cortex_status(Empty())
  # user_id, cluster_id , tenant_id all the same thing :)
  response = await service.all_user_stats(Empty())
  print(response)
  user_id = "6165ba24-e496-491d-86f2-6eb083f24f0b"
  # response = await service.query(QueryRequest(tenants=[user_id], query="up"))
  # response = await service.get_series_metrics(SeriesRequest(tenant=user_id))

  response = await service.extract_raw_series(MatcherRequest(tenant=user_id, match_expr=".+"))
  res =(json.loads(response.data.decode())["data"])

  # channel.close()
  return res

res = await main()
res

UserIDStatsList(items=[UserIDStats(user_id='6165ba24-e496-491d-86f2-6eb083f24f0b', ingestion_rate=11939.420721840364, num_series=139094, api_ingestion_rate=11902.577595792165, rule_ingestion_rate=36.84312604820007)])


{'resultType': 'vector',
 'result': [{'metric': {'__name__': ':node_memory_MemAvailable_bytes:sum',
    '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b'},
   'value': [1678417173.909, '57891811328']},
  {'metric': {'__name__': 'ALERTS',
    '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
    'alertname': 'KubeControllerManagerDown',
    'alertstate': 'firing',
    'severity': 'critical'},
   'value': [1678417173.909, '1']},
  {'metric': {'__name__': 'ALERTS',
    '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
    'alertname': 'KubeSchedulerDown',
    'alertstate': 'firing',
    'severity': 'critical'},
   'value': [1678417173.909, '1']},
  {'metric': {'__name__': 'ALERTS',
    '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
    'alertname': 'Watchdog',
    'alertstate': 'firing',
    'severity': 'none'},
   'value': [1678417173.909, '1']},
  {'metric': {'__name__': 'ALERTS_FOR_STATE',
    '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
    'a

In [9]:
result = res["result"]
result

[{'metric': {'__name__': ':node_memory_MemAvailable_bytes:sum',
   '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b'},
  'value': [1678417173.909, '57891811328']},
 {'metric': {'__name__': 'ALERTS',
   '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
   'alertname': 'KubeControllerManagerDown',
   'alertstate': 'firing',
   'severity': 'critical'},
  'value': [1678417173.909, '1']},
 {'metric': {'__name__': 'ALERTS',
   '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
   'alertname': 'KubeSchedulerDown',
   'alertstate': 'firing',
   'severity': 'critical'},
  'value': [1678417173.909, '1']},
 {'metric': {'__name__': 'ALERTS',
   '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
   'alertname': 'Watchdog',
   'alertstate': 'firing',
   'severity': 'none'},
  'value': [1678417173.909, '1']},
 {'metric': {'__name__': 'ALERTS_FOR_STATE',
   '__tenant_id__': '6165ba24-e496-491d-86f2-6eb083f24f0b',
   'alertname': 'KubeControllerManagerDown',
   'severity': 'cri

In [10]:
len(result)

131120

131120

In [18]:
result1 = [r for r in result if "cpu" in r["metric"]["__name__"]]

In [53]:
len(result1)

1933

In [55]:
s = set()
for r in result1:
    s.add(r["metric"]["__name__"])
s

{'cluster:namespace:pod_cpu:active:kube_pod_container_resource_limits',
 'cluster:namespace:pod_cpu:active:kube_pod_container_resource_requests',
 'cluster:node_cpu:ratio',
 'cluster:node_cpu:ratio_rate5m',
 'cluster:node_cpu:sum_rate5m',
 'container_cpu_cfs_periods_total',
 'container_cpu_cfs_throttled_periods_total',
 'container_cpu_cfs_throttled_seconds_total',
 'container_cpu_load_average_10s',
 'container_cpu_system_seconds_total',
 'container_cpu_usage_seconds_total',
 'container_cpu_user_seconds_total',
 'container_spec_cpu_period',
 'container_spec_cpu_quota',
 'container_spec_cpu_shares',
 'go_memstats_gc_cpu_fraction',
 'instance:node_cpu:rate:sum',
 'instance:node_cpu:ratio',
 'instance:node_cpu_utilisation:rate5m',
 'instance:node_load1_per_cpu:ratio',
 'instance:node_num_cpu:sum',
 'machine_cpu_cores',
 'machine_cpu_physical_cores',
 'machine_cpu_sockets',
 'namespace_cpu:kube_pod_container_resource_limits:sum',
 'namespace_cpu:kube_pod_container_resource_requests:sum',
 '

In [30]:

async def get_user_id(service):
    response = await service.all_user_stats(Empty())
    return response.items[0].user_id

user_id = await get_user_id(service)
user_id

'6165ba24-e496-491d-86f2-6eb083f24f0b'

In [50]:

# query = 'sum(rate(container_cpu_usage_seconds_total{namespace="opni",container!="",pod!=""}[1m])) by (pod)'
query = 'sum(rate(container_cpu_usage_seconds_total{namespace="opni"}[1m])) by (pod)'
response = await service.query(QueryRequest(tenants=[user_id], query=query))
response = json.loads(response.data.decode())["data"]

In [51]:
response["result"]

[{'metric': {'pod': 'opni-prometheus-node-exporter-6zq4r'},
  'value': [1678419786.754, '0.0032273706306036208']},
 {'metric': {'pod': 'etcd-0'},
  'value': [1678419786.754, '0.048333847322082846']},
 {'metric': {'pod': 'opni-kube-prometheus-stack-operator-b6bd94cb-vn2f7'},
  'value': [1678419786.754, '0.0007883594087017497']},
 {'metric': {'pod': 'opni-prometheus-node-exporter-ljdr6'},
  'value': [1678419786.754, '0.001556873181530672']},
 {'metric': {'pod': 'opni-prometheus-node-exporter-d4tmr'},
  'value': [1678419786.754, '0.00481128044618245']},
 {'metric': {'pod': 'opni-agent-0'},
  'value': [1678419786.754, '0.06877475234593705']},
 {'metric': {'pod': 'opni-gateway-799dfbf9cf-txnfh'},
  'value': [1678419786.754, '0.07035532147042549']},
 {'metric': {'pod': 'cortex-all-0'},
  'value': [1678419786.754, '0.43964753528774575']},
 {'metric': {'pod': 'opni-kube-state-metrics-5df8ccf5-mkhfl'},
  'value': [1678419786.754, '0.001547714804513181']},
 {'metric': {'pod': 'my-pod'}, 'value':

In [75]:
query = 'sum(rate(container_cpu_usage_seconds_total{namespace="opni"}[1m])) by (pod)'
from datetime import datetime, timedelta
now = datetime.now()
now_seconds = now.replace(microsecond=0)
start_time = now_seconds - timedelta(minutes=60)
end_time = now_seconds
response = await service.query_range(QueryRangeRequest(tenants=[user_id], query=query, start=start_time, end=end_time, step=timedelta(minutes=1)))
response = json.loads(response.data.decode())["data"]

In [76]:
response

{'resultType': 'matrix',
 'result': [{'metric': {'pod': 'cortex-all-0'},
   'values': [[1678418340, '0.5195668241838232'],
    [1678418400, '0.5220951725694118'],
    [1678418460, '0.652542997497602'],
    [1678418520, '0.7428831273530412'],
    [1678418580, '0.7062573244430748'],
    [1678418640, '0.7649759995485885'],
    [1678418700, '0.5883636129543734'],
    [1678418760, '0.6334703868345836'],
    [1678418820, '0.7193340876881565'],
    [1678418880, '0.6069109725744772'],
    [1678418940, '0.591224693661803'],
    [1678419000, '0.7739165802382061'],
    [1678419060, '0.7088961918870287'],
    [1678419120, '0.4065520812353852'],
    [1678419180, '0.546472013923531'],
    [1678419240, '0.5502018359941648'],
    [1678419300, '0.5784078917618573'],
    [1678419360, '0.6609473451845399'],
    [1678419420, '0.4666519208909586'],
    [1678419480, '0.56066051686322'],
    [1678419540, '0.7530989480334687'],
    [1678419600, '0.4345673048458414'],
    [1678419660, '0.6537888156828715'],
  

2023-03-09 20:17:17
